Init

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
from torchvision import datasets, transforms
import time
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

cuda


Model Definition

In [3]:
class CNN(nn.Module):
    def __init__(self, inChannels=3, chunkSizeX = 64,chunkSizeY = 64):
        super(CNN, self).__init__()
        self.chunkSizeX = chunkSizeX
        self.chunkSizeY = chunkSizeY
        self.intermediateSize = 1024
        self.transfer = nn.Sigmoid()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=3*chunkSizeX*chunkSizeY, out_features=self.intermediateSize),
            nn.ReLU(),
        )                
        self.flatten = nn.Flatten()  
        self.unflatten  = nn.Unflatten(1, (3,chunkSizeY,chunkSizeX))

        self.decoder = nn.Sequential(
            nn.Linear(in_features=self.intermediateSize, out_features=3*chunkSizeX*chunkSizeY),
            self.transfer,
        )
    
    def forward(self, x):
        out = self.flatten(x)
        out = self.encoder(out)
        out = self.decoder(out)
        out = self.unflatten(out)
        return out

model = CNN().to(device)

In [4]:
# Create Data Set
trainImage = Image.open("./test-cropped.png")
chunkSizeX = model.chunkSizeX
chunkSizeY = model.chunkSizeY
trainImageTensor = torch.tensor(np.array(trainImage)).to(device)/255.
print("Image Tensor Shape: " + str(trainImageTensor.shape))
width = trainImageTensor.shape[1]
height = trainImageTensor.shape[0]
print("Image size: " + str(width) + "x" + str(height))
xRemain = width % chunkSizeX
yRemain = height % chunkSizeY
print("Remains: " + str(xRemain) + " - " + str(yRemain))
trainImageTensor = trainImageTensor[0:height-yRemain, 0:width-xRemain,:]
width = trainImageTensor.shape[1]
height = trainImageTensor.shape[0]
print("trainImageTensor shape after crop= ",trainImageTensor.shape)
print("New Image size: " + str(width) + "x" + str(height))

xChunks = int((width)/chunkSizeX)
yChunks = int((height)/chunkSizeY)
chunks = xChunks * yChunks
print("xChunks: " + str(xChunks))
print("yChunks: " + str(yChunks))
print("Chunks: " + str(chunks))
stride4 = 1
stride3 = 3
stride2 = width*3
stride1 = chunkSizeX
stride0 = chunkSizeY*width*3
trainImageChunks = trainImageTensor.as_strided((yChunks, xChunks,chunkSizeY, chunkSizeX,3), (stride0,stride1, stride2, stride3, stride4)).flatten(0,1)
print(trainImageChunks.shape)
print("chunk shape = ",trainImageChunks[0].shape)

# for i in range(0, 32):
#     chunk = trainImageChunks[8192-32+i].cpu()
#     img = Image.fromarray(np.uint8(chunk*255)).save("./images/test-out_"+str(i)+".png")


Image Tensor Shape: torch.Size([2048, 4096, 3])
Image size: 4096x2048
Remains: 0 - 0
trainImageTensor shape after crop=  torch.Size([2048, 4096, 3])
New Image size: 4096x2048
xChunks: 64
yChunks: 32
Chunks: 2048
torch.Size([2048, 64, 64, 3])
chunk shape =  torch.Size([64, 64, 3])


In [8]:
#model.load_state_dict(torch.load("./model2.pth"))
#model.eval()

#optimizer = optim.Adam(model.parameters(), lr=0.005)
optimizer = optim.SGD(model.parameters(), lr=10, momentum=0.1)
writer = SummaryWriter()
# create the loss function
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
# train the model
batchSize = 32
epochs = 5000
batchCount = int(chunks/batchSize)
print ("batchCount = ",batchCount)
for epoch in range(1, epochs+1):
    for batch in range(0, batchCount):
        batchStart = batch * batchSize
        batchEnd = batchStart + batchSize
        if(batchEnd > chunks):
            batchEnd = chunks
        batchTensor = trainImageChunks[batchStart:batchEnd].transpose(1,3).transpose(2,3)
        optimizer.zero_grad()
        output = model(batchTensor)
        loss = criterion(output, batchTensor)
        loss.backward()
        optimizer.step()
        #print("Batch: " + str(batch) + " Loss: " + str(loss.item()))    
    print("Epoch: " + str(epoch) + " Loss: " + str(loss.item()))    
    inputExamples = batchTensor[0:10,:,:,:]
    outputExamples = output[0:10,:,:,:]
    inputGrid = torchvision.utils.make_grid(inputExamples)
    outputGrid = torchvision.utils.make_grid(outputExamples)
    writer.add_scalar('Loss/train', loss.item(), epoch)
    writer.add_image('input', inputGrid, epoch)
    writer.add_image('output', outputGrid, epoch)
    writer.flush()

# save the model
torch.save(model.state_dict(), "./model6.pth")


batchCount =  64
Epoch: 1 Loss: 8.155219256877899e-05
Epoch: 2 Loss: 8.160957077052444e-05
Epoch: 3 Loss: 8.147535845637321e-05
Epoch: 4 Loss: 8.133734809234738e-05
Epoch: 5 Loss: 8.069014438660815e-05
Epoch: 6 Loss: 8.052380144363269e-05
Epoch: 7 Loss: 8.019640517886728e-05
Epoch: 8 Loss: 8.030976459849626e-05
Epoch: 9 Loss: 8.015418279683217e-05
Epoch: 10 Loss: 8.023341069929302e-05
Epoch: 11 Loss: 8.022936526685953e-05
Epoch: 12 Loss: 7.998707587830722e-05
Epoch: 13 Loss: 7.981267117429525e-05
Epoch: 14 Loss: 7.975767221068963e-05
Epoch: 15 Loss: 7.952728628879413e-05
Epoch: 16 Loss: 7.925247336970642e-05
Epoch: 17 Loss: 7.940196519484743e-05
Epoch: 18 Loss: 7.935997564345598e-05
Epoch: 19 Loss: 7.908565748948604e-05
Epoch: 20 Loss: 7.894516602391377e-05
Epoch: 21 Loss: 7.895624730736017e-05
Epoch: 22 Loss: 7.943593664094806e-05
Epoch: 23 Loss: 7.92135251685977e-05
Epoch: 24 Loss: 7.91036436567083e-05
Epoch: 25 Loss: 7.90557824075222e-05
Epoch: 26 Loss: 7.963548705447465e-05
Epoch: 